In [31]:
import pandas as pd 
from pandas.tseries.offsets import BDay
from datetime import datetime, timedelta
from dateutil import relativedelta
import calendar

import sys
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases

In [32]:
conexao = Connect.connect_techdb()
posicoes = Connect.import_table(conexao, "posicao")
proventos = Connect.import_table(conexao, 'proventos_futuros_rf')
conexao.close()

In [33]:
def vencimentos_semana():
    ## Adicionar os vencimentos nos próximos 7 dias
    sete_dias = (datetime.today() + timedelta(days=6)).strftime("%Y-%m-%d")
    vencimento_semana = posicoes[posicoes['VENCIMENTO'] <= sete_dias]
    vencimento_semana = vencimento_semana[vencimento_semana['Mercado'] != "Valor em Trânsito"]
    vencimento_semana = vencimento_semana.loc[:,["Conta", "Nome", "Assessor", "Valor Líquido"]]
    vencimento_semana = vencimento_semana.groupby(["Conta", "Nome", "Assessor"]).sum()['Valor Líquido'].reset_index()
    vencimento_semana.rename(columns={'Valor Líquido':"Vencimentos na Semana"}, inplace=True)
    vencimento_semana['Nome'] = [nome.upper() for nome in vencimento_semana['Nome']]
    return vencimento_semana

In [34]:
def vencimentos_mes_atual():
    ## Adicionar os vencimentos nos próximos 7 dias
    primeiro_dia = datetime.today().strftime("%Y-%m-01")
    hoje = datetime.today()
    ultimo_dia = calendar.monthrange(hoje.year, hoje.month)[1] 
    data_ultimo_dia = hoje.replace(day=ultimo_dia)
    vencimento_prox_mes= posicoes[(posicoes['VENCIMENTO'] >= primeiro_dia) &
                                 (posicoes['VENCIMENTO'] <= data_ultimo_dia)]
    vencimento_prox_mes = vencimento_prox_mes[vencimento_prox_mes['Mercado'] != "Valor em Trânsito"]
    vencimento_prox_mes = vencimento_prox_mes.loc[:,["Conta", "Nome", "Assessor", "Valor Líquido"]]
    vencimento_prox_mes = vencimento_prox_mes.groupby(["Conta", "Nome", "Assessor"]).sum()['Valor Líquido'].reset_index()
    vencimento_prox_mes.rename(columns={'Valor Líquido':"Vencimentos no Mês"}, inplace=True)
    return vencimento_prox_mes

In [35]:
vencimentos_semana_atual = vencimentos_semana()

In [36]:
vencimentos_mes_proximo = vencimentos_mes_atual()

In [37]:
def proventos_semana(proventos_df):
    sete_dias = (datetime.today() + timedelta(days=6)).strftime("%Y-%m-%d")
    # Filtra >= Hoje e <= Hoje+6
    proventos_df = proventos_df[(proventos_df['Data pagamento'] <= sete_dias) & 
                                (proventos_df['Data pagamento'] >= datetime.today().strftime("%Y-%m-%d"))].loc[:,["Conta", "Nome", "Assessor", "Total proventos"]]
    proventos_df = proventos_df.groupby(["Conta", "Nome", "Assessor"]).sum().reset_index()
    proventos_df.rename(columns={"Total proventos":"Proventos na semana"}, inplace=True)
    return proventos_df

In [38]:
def proventos_mes_atual(proventos_df):
    hoje = datetime.today()
    ultimo_dia = calendar.monthrange(hoje.year, hoje.month)[1] 
    data_ultimo_dia = hoje.replace(day=ultimo_dia)
    proventos_df = proventos_df[(proventos_df['Data pagamento'] <= data_ultimo_dia) & (proventos_df['Data pagamento'] >= datetime.today())].loc[:,["Conta", "Nome", "Assessor", "Total proventos"]]
    proventos_df = proventos_df.groupby(["Conta", "Nome", "Assessor"]).sum().reset_index()
    proventos_df.rename(columns={"Total proventos":"Proventos no Mês"}, inplace=True)
    return proventos_df

In [39]:
proventos_mes = proventos_mes_atual(proventos)

In [40]:
proventos_futuros = proventos_semana(proventos)

In [41]:
vencimentos_proventos_mes = proventos_mes.merge(vencimentos_mes_proximo, on=['Conta', 'Nome', 'Assessor'], how='outer')

In [42]:
vencimentos_proventos_semana = proventos_futuros.merge(vencimentos_semana_atual, on=['Conta', 'Nome', 'Assessor'], how='outer')

In [43]:
conexao = Connect.connect_techdb()
times = Connect.import_table(conexao, "times_nova_empresa")
conexao.close()

In [44]:
for assessor in times['Assessor']:
    sem_assessor = vencimentos_proventos_semana[vencimentos_proventos_semana['Assessor'] == assessor]
    if len(sem_assessor) > 0:
        sem_assessor.to_excel("./assessores_semana/{}_vencimentos_proventos_semana_atual.xlsx".format(assessor, assessor), header=True, index=False)

In [45]:
for assessor in times['Assessor']:
    mes_assessor = vencimentos_proventos_mes[vencimentos_proventos_mes['Assessor'] == assessor]
    if len(mes_assessor) > 0:
        mes_assessor.to_excel("./assessores_mes/{}_vencimentos_proventos_mes_atual.xlsx".format(assessor, assessor), header=True, index=False)